In [ ]:
#Weight & Biases
import wandb

#Yaml
from yaml import load, FullLoader

#Embeddings
from dvats.all import *
from tsai.data.preparation import prepare_forecasting_data
from tsai.data.validation import get_forecasting_splits
from fastcore.all import *

#Dimensionality reduction
from tsai.imports import *

#Clustering
import hdbscan
import utils.config as cfg_
import time
import seaborn as sns

In [ ]:
#Get W&B API
api = wandb.Api()

In [ ]:
import os
path = os.path.expanduser("~/work/nbs_pipeline/")
name="05-xai-lrp-TEST"
runname = name
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"

In [ ]:
config_lrp = cfg_.get_artifact_config_xai_lrp(False)
cfg_.show_attrdict(config_lrp)

use_wandb: True
wandb_group: None
wandb_entity: mi-santamaria
wandb_project: deepvats
emb_artifact: mi-santamaria/deepvats/embeddings-mvp-SWV:latest
job_type: layer_relevance_propagation
allow_val_change: True


In [ ]:
config_lrp.use_wandb

True

In [ ]:
run_lrp = wandb.init(
    entity           = config_lrp.wandb_entity,
    project          = config_lrp.wandb_project if config_lrp.use_wandb else 'work-nbs', 
    group            = config_lrp.wandb_group,
    allow_val_change = config_lrp.allow_val_change, 
    job_type         = config_lrp.job_type, 
    mode             = 'online' if config_lrp.use_wandb else 'disabled',
    anonymous        = 'never' if config_lrp.use_wandb else 'must',
    config           =  config_lrp,
    resume           = 'allow',
    name = runname
)
config_lrp = wandb.config # Object for storing hyperparameters

wandb: Currently logged in as: mi-santamaria. Use `wandb login --relogin` to force relogin


In [ ]:
embs, emb_artifact, emb_config = get_embeddings(config_lrp, run_lrp, api, True)

embeddings-mvp-SWV:v0


wandb:   1 of 1 files downloaded.  


In [ ]:
embs.shape

(520, 128)

In [ ]:
cfg_.show_attrdict(emb_config)

cpu: False
input_ar: mi-santamaria/deepvats/toy:v2
use_wandb: True
wandb_group: embeddings
enc_artifact: mi-santamaria/deepvats/mvp-SWV:latest
wandb_entity: mi-santamaria
wandb_project: deepvats


In [ ]:
#| export
def get_dataset_old(config, run_dr, config_dr, api, print_flag = False):
    # Botch to use artifacts offline
    artifacts_gettr = run_dr.use_artifact if config_dr.use_wandb else api.artifact
    enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')
    if print_flag: print (enc_artifact.name)
    ## TODO: This only works when you run it two timeS! WTF?
    try:
        enc_learner = enc_artifact.to_obj()
    except:
        enc_learner = enc_artifact.to_obj()
    #enc_artifact.metadata
    ## Restore artifact
    enc_logger = enc_artifact.logged_by()
    enc_artifact_train = artifacts_gettr(enc_logger.config['train_artifact'], type='dataset')
    if enc_logger.config['valid_artifact'] is not None:
        enc_artifact_valid = artifacts_gettr(enc_logger.config['valid_artifact'], type='dataset')
        if print_flag: print("enc_artifact_valid:", enc_artifact_valid.name)
    if print_flag: print("enc_artifact_train: ", enc_artifact_train.name)
    if config_dr.dr_artifact is not None:
        dr_artifact = artifacts_gettr(config_dr.dr_artifact)
    else:
        dr_artifact = enc_artifact_train
    if print_flag: print("DR artifact train: ", dr_artifact.name)
    df = dr_artifact.to_df()
    if print_flag: display(df.head())
    return df, dr_artifact, enc_artifact, enc_learner

In [ ]:
#| export
def get_dataset(
    config_lrp, 
    config_emb,
    config_dr,
    run_lrp,
    api, 
    print_flag = False
):
    # Botch to use artifacts offline
    artifacts_gettr = run_lrp.use_artifact if config_lrp.use_wandb else api.artifact
    enc_artifact = artifacts_gettr(config_emb['enc_artifact'], type='learner')
    if print_flag: print (enc_artifact.name)
    ## TODO: This only works when you run it two timeS! WTF?
    try:
        enc_learner = enc_artifact.to_obj()
    except:
        enc_learner = enc_artifact.to_obj()
    
    ## Restore artifact
    enc_logger = enc_artifact.logged_by()
    enc_artifact_train = artifacts_gettr(enc_logger.config['train_artifact'], type='dataset')
    #cfg_.show_attrdict(enc_logger.config)
    if enc_logger.config['valid_artifact'] is not None:
        enc_artifact_valid = artifacts_gettr(enc_logger.config['valid_artifact'], type='dataset')
        if print_flag: print("enc_artifact_valid:", enc_artifact_valid.name)
    if print_flag: print("enc_artifact_train: ", enc_artifact_train.name)
    
    if config_dr['dr_artifact'] is not None:
        print("Is not none")
        dr_artifact = artifacts_gettr(config_dr['enc_artifact'])
    else:
        dr_artifact = enc_artifact_train
    if print_flag: print("DR artifact train: ", dr_artifact.name)
    if print_flag: print("--> DR artifact name", dr_artifact.name)
    dr_artifact
    df = dr_artifact.to_df()
    if print_flag: print("--> DR After to df", df.shape)
    if print_flag: display(df.head())
    return df, dr_artifact, enc_artifact, enc_learner

In [ ]:
cfg_.show_attrdict(emb_config)

cpu: False
input_ar: mi-santamaria/deepvats/toy:v2
use_wandb: True
wandb_group: embeddings
enc_artifact: mi-santamaria/deepvats/mvp-SWV:latest
wandb_entity: mi-santamaria
wandb_project: deepvats


In [ ]:
config_dr, _ = cfg_.get_artifact_config_dimensionality_reduction(False)

In [ ]:
cfg_.show_attrdict(config_dr)

use_wandb: True
wandb_group: None
wandb_entity: mi-santamaria
wandb_project: deepvats
valid_artifact: None
train_artifact: mi-santamaria/deepvats/mvp-SWV:latest
dr_artifact: None
n_neighbors: 15
min_dist: 0.1
random_state: 1234
metric: euclidean


In [ ]:
df, dr_artifact, enc_artifact, enc_learner = get_dataset(
    config_lrp,
    emb_config,
    config_dr,
    run_lrp,
    api, 
    True
)

mvp-SWV:v66


wandb:   1 of 1 files downloaded.  


enc_artifact_train:  toy:v2
DR artifact train:  toy:v2
--> DR artifact name toy:v2


wandb:   1 of 1 files downloaded.  


--> DR After to df (550, 3)


,T3,T2,T1
1970-01-01 00:00:00,0.741822,0.637180,0.565117
1970-01-01 00:00:01,0.739731,0.629415,0.493513
1970-01-01 00:00:02,0.718757,0.539220,0.469350
1970-01-01 00:00:03,0.730169,0.577670,0.444100
1970-01-01 00:00:04,0.752406,0.570180,0.373008


In [ ]:
artifacts_gettr = run_lrp.use_artifact if config_lrp.use_wandb else api.artifact

In [ ]:
enc_artifact = artifacts_gettr(emb_config['enc_artifact'], type='learner')

In [ ]:
enc_artifact.name

'mvp-SWV:v66'

In [ ]:
embs

array([[0.33592895, 0.        , 0.18003725, ..., 0.03459276, 2.0539532 ,
        0.02495872],
       [0.27895513, 0.        , 0.23209466, ..., 0.02376847, 2.166164  ,
        0.03464536],
       [0.25264975, 0.        , 0.27396208, ..., 0.02094788, 2.2761836 ,
        0.0396877 ],
       ...,
       [0.7967689 , 3.29659   , 3.7586546 , ..., 0.03084148, 0.        ,
        6.3082776 ],
       [0.79666084, 3.2738378 , 3.7270284 , ..., 0.02614438, 0.        ,
        6.2277656 ],
       [0.8129102 , 3.2375443 , 3.6983783 , ..., 0.04231085, 0.        ,
        6.1111894 ]], dtype=float32)

In [ ]:
config_dr

({'use_wandb': True,
  'wandb_group': None,
  'wandb_entity': 'mi-santamaria',
  'wandb_project': 'deepvats',
  'valid_artifact': None,
  'train_artifact': 'mi-santamaria/deepvats/mvp-SWV:latest',
  'n_neighbors': 15,
  'min_dist': 0.1,
  'random_state': 1234,
  'metric': 'euclidean'},
 'dimensionality_reduction')

In [ ]:
df, dr_artifact, enc_artifact, enc_learner = get_dataset(
    config_dr, 
    run_lrp, 
    emb_config, 
    

In [ ]:
enc_artifact = artifacts_gettr(emb_config['enc_artifact'], type='learner')

NameError: name 'artifacts_gettr' is not defined

In [ ]:
enc_artifact

<Artifact QXJ0aWZhY3Q6NzExOTI4MjQ5>

In [ ]:
enc_logger = enc_artifact.logged_by()

AttributeError: '<Run mi-santamaria/deepvats/ad865x5t (finished)>' object has no attribute 'logged_by'

In [ ]:
enc_learner = enc_learner.logged_by()

NameError: name 'enc_learner' is not defined

In [ ]:
def get_embeddings(config, run_dr, config_dr, api, print_flag = False):
    # Botch to use artifacts offline
    artifacts_gettr = run_dr.use_artifact if config_dr.use_wandb else api.artifact
    enc_artifact = artifacts_gettr(config.emb_artifact, type='object')
    if print_flag: print (emb_artifact.name)
    ## TODO: This only works when you run it two timeS! WTF?
    try:
        enc_learner = enc_artifact.to_obj()
    except:
        enc_learner = enc_artifact.to_obj()
    #enc_artifact.metadata
    ## Restore artifact
    enc_logger = enc_artifact.logged_by()
    enc_artifact_train = artifacts_gettr(enc_logger.config['train_artifact'], type='dataset')
    if enc_logger.config['valid_artifact'] is not None:
        enc_artifact_valid = artifacts_gettr(enc_logger.config['valid_artifact'], type='dataset')
        if print_flag: print("enc_artifact_valid:", enc_artifact_valid.name)
    if print_flag: print("enc_artifact_train: ", enc_artifact_train.name)
    if config_dr.dr_artifact is not None:
        dr_artifact = artifacts_gettr(config_dr.dr_artifact)
    else:
        dr_artifact = enc_artifact_train
    if print_flag: print("DR artifact train: ", dr_artifact.name)
    df = dr_artifact.to_df()
    if print_flag: display(df.head())
    return df, dr_artifact, enc_artifact, enc_learner